# Spatial Interaction Model (Enhancements)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import numpy as np
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib import rcParams
from math import sqrt
import scipy.stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Load employment data for 2011 for districts
df = pd.read_csv('wed.csv', index_col=[0])
df = df.loc[df['Distance'] != 0]
df.reset_index(inplace=True)
df.drop(columns={'index','Day', 'Ourban', 'Durban', 'OInc', 'DInc', 'OrigCode', 'DestCode'}, inplace=True)
df.head()

,Trips,Origin,Destination,OPop,DEmpl,Distance
0,10728,Gamle Oslo,Grünerløkka,41736,29560,2143.111352
1,3820,Gamle Oslo,Sagene,41736,19408,3837.411674
2,7288,Gamle Oslo,St. Hanshaugen,41736,54104,4360.828507
3,6342,Gamle Oslo,Frogner,41736,48082,5157.307531
4,2404,Gamle Oslo,Ullern,41736,38334,5495.850695


In [3]:
#take the variables and produce logarithms of them
x_variables = ["OPop", "DEmpl", "Distance"]
log_x_vars = []
for x in x_variables:
    df[f"log_{x}"] = np.log(df[x])
    log_x_vars.append(f"log_{x}")

## Origin/Production Constrained Model

In [5]:
#Singly- Production (Origin) Constrained
#create the formula (the "-1" indicates no intercept in the regression model).
formula = 'Trips ~ Origin + log_DEmpl + log_Distance-1'
#run a production constrained sim
prodSIM = smf.glm(formula = formula, data=df, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(prodSIM.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Trips   No. Observations:                  272
Model:                            GLM   Df Residuals:                      253
Model Family:                 Poisson   Df Model:                           18
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -46809.
Date:                Sun, 20 Mar 2022   Deviance:                       91169.
Time:                        01:36:31   Pearson chi2:                 9.77e+04
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Origin[Alna]          

In [6]:
#to create O_i, take df ...then... group by origcodenew ...then... summarise by calculating the sum of Total
O_i = pd.DataFrame(df.groupby(["Origin"])["Trips"].agg(np.sum))
O_i.rename(columns={"Trips":"O_i"}, inplace = True)
df = df.merge(O_i, on = "Origin", how = "left" )

D_j = pd.DataFrame(df.groupby(["Destination"])["Trips"].agg(np.sum))
D_j.rename(columns={"Trips":"D_j"}, inplace = True)
df = df.merge(D_j, on = "Destination", how = "left" )

#Now we want to fish the coefficients out of the prodSim glm object, by pulling out the parameter values
coefs = pd.DataFrame(prodSIM.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(Origin)", "\[", "\]"]
for x in to_repl: coefs["coef"] = coefs["coef"].str.replace(x, "")
coefs.head()

,coef,alpha_i
0,Alna,14.734068
1,Bjerke,14.257883
2,Frogner,14.895574
3,Gamle Oslo,14.591050
4,Grorud,14.073215


In [7]:
#then once you have done this you can join them back into the dataframes
df = df.merge(coefs, left_on="Origin", right_on="coef", how = "left")
df.drop(columns = ["coef"], inplace = True)
df.head()

,Trips,Origin,Destination,OPop,DEmpl,Distance,log_OPop,log_DEmpl,log_Distance,O_i,D_j,alpha_i
0,10728,Gamle Oslo,Grünerløkka,41736,29560,2143.111352,10.639119,10.294177,7.670014,69093,68101,14.59105
1,3820,Gamle Oslo,Sagene,41736,19408,3837.411674,10.639119,9.873441,8.252553,69093,47062,14.59105
2,7288,Gamle Oslo,St. Hanshaugen,41736,54104,4360.828507,10.639119,10.898663,8.380417,69093,75747,14.59105
3,6342,Gamle Oslo,Frogner,41736,48082,5157.307531,10.639119,10.780663,8.548170,69093,73990,14.59105
4,2404,Gamle Oslo,Ullern,41736,38334,5495.850695,10.639119,10.554093,8.611749,69093,34408,14.59105


In [8]:
#To generate our trip generation estimates
alpha_i = prodSIM.params[0:17]
gamma = prodSIM.params[17]
beta = prodSIM.params[18]

df["prodsimest1"] = np.exp(df["alpha_i"]+gamma*df["log_DEmpl"] + beta*df["log_Distance"])
df.head(10)

,Trips,Origin,Destination,OPop,DEmpl,Distance,log_OPop,log_DEmpl,log_Distance,O_i,D_j,alpha_i,prodsimest1
0,10728,Gamle Oslo,Grünerløkka,41736,29560,2143.111352,10.639119,10.294177,7.670014,69093,68101,14.59105,11101.626901
1,3820,Gamle Oslo,Sagene,41736,19408,3837.411674,10.639119,9.873441,8.252553,69093,47062,14.59105,4975.480663
2,7288,Gamle Oslo,St. Hanshaugen,41736,54104,4360.828507,10.639119,10.898663,8.380417,69093,75747,14.59105,6060.531414
3,6342,Gamle Oslo,Frogner,41736,48082,5157.307531,10.639119,10.780663,8.548170,69093,73990,14.59105,4815.012543
4,2404,Gamle Oslo,Ullern,41736,38334,5495.850695,10.639119,10.554093,8.611749,69093,34408,14.59105,4152.794051
5,1895,Gamle Oslo,Vestre Aker,41736,13362,9036.710976,10.639119,9.500170,9.109051,69093,34620,14.59105,1659.465013
6,4609,Gamle Oslo,Nordre Aker,41736,42238,5531.682962,10.639119,10.651076,8.618247,69093,62424,14.59105,4258.015246
7,3122,Gamle Oslo,Bjerke,41736,22537,4888.176112,10.639119,10.022914,8.494575,69093,35168,14.59105,3972.685071
8,1384,Gamle Oslo,Grorud,41736,9297,10092.272151,10.639119,9.137447,9.219525,69093,19460,14.59105,1296.608008
9,1522,Gamle Oslo,Stovner,41736,9490,10711.853108,10.639119,9.157994,9.279106,69093,18875,14.59105,1220.073727


In [4]:
#Calculate Root Mean Square Error between a series of observed and estimated values
#    Observed: Series of actual observed values
#    estimated: Series of predicted values
def CalcRSqaured(observed, estimated):
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    return R2
def CalcRMSE(observed, estimated):
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    return RMSE

In [9]:
#first round the estimates
df["prodsimest1"] = round(df["prodsimest1"],0)
CalcRSqaured(df["Trips"], df["prodsimest1"])

0.8436015570342156

In [10]:
CalcRMSE(df["Trips"], df["prodsimest1"])

1137.748

##  Destination/Attraction Constrained Model

In [11]:
#Singly - Destination or Attraction Constrained
#create the formula (the "-1" indicates no intercept in the regression model).
attr_form = 'Trips ~ Destination + log_OPop + log_Distance-1'
#run a attraction constrained sim
attrSim = smf.glm(formula = attr_form, data=df, family=sm.families.Poisson()).fit()
print(attrSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Trips   No. Observations:                  272
Model:                            GLM   Df Residuals:                      253
Model Family:                 Poisson   Df Model:                           18
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -65492.
Date:                Sun, 20 Mar 2022   Deviance:                   1.2854e+05
Time:                        01:36:32   Pearson chi2:                 1.23e+05
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Destination[

In [12]:
#get the predictions
predictions = attrSim.get_prediction(df[["Destination", "log_OPop", "log_Distance"]])
predictions_summary_frame = predictions.summary_frame()
df["attrsimFitted"] = round(predictions_summary_frame["mean"],0)
CalcRSqaured(df["Trips"], df["attrsimFitted"])

0.7540288373047039

In [13]:
CalcRMSE(df["Trips"], df["attrsimFitted"])

1444.73

## Doubly Constrained Model

In [14]:
dbl_form = 'Trips ~ Destination + Origin + log_Distance-1'
#run a production constrained sim
doubSim = smf.glm(formula = dbl_form, data=df, family=sm.families.Poisson()).fit()
print(doubSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Trips   No. Observations:                  272
Model:                            GLM   Df Residuals:                      238
Model Family:                 Poisson   Df Model:                           33
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -27618.
Date:                Sun, 20 Mar 2022   Deviance:                       52787.
Time:                        01:36:32   Pearson chi2:                 5.34e+04
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Destination[

In [15]:
#get the estimates
df["doubsimfitted"] = np.round(doubSim.predict())
CalcRSqaured(df["Trips"], df["doubsimfitted"])

0.8985709730777629

In [16]:
CalcRMSE(df["Trips"], df["doubsimfitted"])

920.326

### Distance Decay Method

In [17]:
# Run a production constrained SIM with a negative exponential cost function.
doubsim_form = 'Trips ~ Origin + Destination + Distance -1'
doubsim1 = smf.glm(formula=doubsim_form, data = df, family = sm.families.Poisson()).fit()
print(doubsim1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Trips   No. Observations:                  272
Model:                            GLM   Df Residuals:                      238
Model Family:                 Poisson   Df Model:                           33
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -22384.
Date:                Sun, 20 Mar 2022   Deviance:                       42319.
Time:                        01:36:32   Pearson chi2:                 4.49e+04
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Origin[A

In [18]:
df["doubsimfitted1"] = np.round(doubsim1.predict(),0)
CalcRSqaured(df["Trips"],df["doubsimfitted1"])

0.9313659174952538

In [19]:
CalcRMSE(df["Trips"], df["doubsimfitted1"])

743.693

### Entropy maximisation method

In [20]:
def balance_doubly_constrained(pd, orig_field, dest_field, Oi_field, Dj_field, cij_field, beta, 
                               cost_function, Ai_name = "Ai_new", Bj_name = "Bj_new", converge=0.001):
    # Define some variables
    Oi = pd[[orig_field, Oi_field]]
    Dj = pd[[dest_field,Dj_field]]    
    if cost_function.lower() in ['power','pow']:
        beta_cij = np.exp(beta * np.log(pd[cij_field]))
    elif cost_function.lower() in ['exponential','exp']:
        beta_cij = np.exp(beta * pd[cij_field])
    else:
        return "Cost function not specified properly, use 'exp' or 'pow'"
    
    # Create some helper variables
    cnvg = 1
    iteration = 0
    # Now iteratively rebalance the Ai and Bj terms until convergence
    while cnvg > converge:
        if iteration == 0:
            # This first condition sets starting values for Ai and Bj
            # NB sets starting value of Ai assuming Bj is a vector of 1s.
            # We've already established beta_cij with the appropriate cost function, so...
            Oi = Oi.assign(Ai = Dj[Dj_field] * beta_cij)
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Now set up Bjs using starting values of Ai
            Dj = Dj.assign(Bj = Oi['Ai_old'] * Oi[Oi_field] * beta_cij)
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Increment loop
            iteration += 1
        else:
            # This bit is the iterated bit of the loop which refines the values of Ai and Bj
            # First Ai
            Oi['Ai'] = Dj['Bj_old'] * Dj[Dj_field] * beta_cij
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Drop temporary Ai
            Oi.drop('Ai', axis=1, inplace=True)
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True)
            # Calculate the difference between old and new Ais
            Oi['diff'] = np.absolute((Oi['Ai_old'] - Oi['Ai'])/Oi['Ai_old'])
            # Set new Ais to Ai_old
            Oi['Ai_old'] = Oi['Ai']
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Then Bj
            Dj['Bj'] = Oi['Ai_old'] * Oi[Oi_field] * beta_cij
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Drop temporary Bj
            Dj.drop('Bj', axis=1, inplace=True)
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True)
            # Calculate the difference between old and new Bjs
            Dj['diff'] = np.absolute((Dj['Bj_old'] - Dj['Bj'])/Dj['Bj_old'])
            # Set new Bjs to Bj_old
            Dj['Bj_old'] = Dj['Bj']
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Assign higher sum difference from Ai or Bj to cnvg
            cnvg = np.maximum(Oi['diff'].sum(),Dj['diff'].sum())
            
            # Print and increment loop
            print("Iteration:", iteration)
            iteration += 1

    # When the while loop finishes add the computed Ai_old and Bj_old to the dataframe and return
    pd[Ai_name] = Oi['Ai_old']
    pd[Bj_name] = Dj['Bj_old']
    return pd

In [21]:
#Using the function above we can calculate $A_{i}$ and $B_{j}$ for the previous Poisson model by plugging in the estimate of beta that we generated.
# Use the beta we got from the inverse power model
beta = doubSim.params[-1]
# Get the balancing factors.
df = balance_doubly_constrained(df,'Origin','Destination','O_i','D_j','Distance',beta,'power')

# Now predict the model again using the new Ai and Dj fields.
df['SIM_est_pow'] = np.round(df['O_i'] * df['Ai_new'] * df['D_j'] * df['Bj_new'] *np.exp(np.log(df['Distance'])*beta))
# The matrix
#pd.pivot_table(df,values='SIM_est_pow',index ='Origin',columns='Destination',fill_value=0,aggfunc=sum,margins=True)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [22]:
# Use the beta we got from the negative exponential model
beta = doubsim1.params[-1]
# Get the balancing factors. NB Setting of new field names for Ai and Bj.
df = balance_doubly_constrained(df,'Origin','Destination','O_i','D_j','Distance',beta,'exponential','Ai_exp','Bj_exp')

# Now predict the model again using the new Ai and Dj fields.
df['SIM_est_exp'] = np.round(df['O_i'] * df['Ai_exp'] * df['D_j'] * df['Bj_exp'] * np.exp(df['Distance']*beta))
# Check out the matrix
#pd.pivot_table(df,values='SIM_est_exp',index ='Origin',columns='Destination',fill_value=0,aggfunc=sum,margins=True)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [23]:
CalcRSqaured(df["Trips"],df["SIM_est_exp"])

0.9313659174952538

In [24]:
CalcRMSE(df["Trips"], df["SIM_est_exp"])

743.693

## Scenario Simulations

In [25]:
print(beta)

-0.00023260829135309067


In [26]:
df.head()

,Trips,Origin,Destination,OPop,DEmpl,Distance,log_OPop,log_DEmpl,log_Distance,O_i,...,prodsimest1,attrsimFitted,doubsimfitted,doubsimfitted1,Ai_new,Bj_new,SIM_est_pow,Ai_exp,Bj_exp,SIM_est_exp
0,10728,Gamle Oslo,Grünerløkka,41736,29560,2143.111352,10.639119,10.294177,7.670014,69093,...,11102.0,11043.0,11400.0,8995.0,0.088423,0.705430,11400.0,0.000004,0.732120,8995.0
1,3820,Gamle Oslo,Sagene,41736,19408,3837.411674,10.639119,9.873441,8.252553,69093,...,4975.0,3341.0,3197.0,4087.0,0.088423,0.619157,3197.0,0.000004,0.713809,4087.0
2,7288,Gamle Oslo,St. Hanshaugen,41736,54104,4360.828507,10.639119,10.898663,8.380417,69093,...,6061.0,5488.0,5920.0,6821.0,0.088423,0.843713,5920.0,0.000004,0.836060,6821.0
3,6342,Gamle Oslo,Frogner,41736,48082,5157.307531,10.639119,10.780663,8.548170,69093,...,4815.0,5332.0,5510.0,6403.0,0.088423,1.003974,5510.0,0.000004,0.967071,6403.0
4,2404,Gamle Oslo,Ullern,41736,38334,5495.850695,10.639119,10.554093,8.611749,69093,...,4153.0,2798.0,2554.0,3091.0,0.088423,1.088645,2554.0,0.000004,1.086014,3091.0


In [27]:
df.columns

Index(['Trips', 'Origin', 'Destination', 'OPop', 'DEmpl', 'Distance',
       'log_OPop', 'log_DEmpl', 'log_Distance', 'O_i', 'D_j', 'alpha_i',
       'prodsimest1', 'attrsimFitted', 'doubsimfitted', 'doubsimfitted1',
       'Ai_new', 'Bj_new', 'SIM_est_pow', 'Ai_exp', 'Bj_exp', 'SIM_est_exp'],
      dtype='object')

In [29]:
data= df.drop(columns={'log_OPop', 'log_DEmpl', 'log_Distance', 'O_i', 'D_j', 'alpha_i',
       'prodsimest1', 'attrsimFitted', 'doubsimfitted', 'doubsimfitted1',
       'Ai_new', 'Bj_new', 'SIM_est_pow', 'Ai_exp', 'Bj_exp', 'SIM_est_exp'})
data.head()

,Trips,Origin,Destination,OPop,DEmpl,Distance
0,10728,Gamle Oslo,Grünerløkka,41736,29560,2143.111352
1,3820,Gamle Oslo,Sagene,41736,19408,3837.411674
2,7288,Gamle Oslo,St. Hanshaugen,41736,54104,4360.828507
3,6342,Gamle Oslo,Frogner,41736,48082,5157.307531
4,2404,Gamle Oslo,Ullern,41736,38334,5495.850695


### Cost of transport increases significantly (Fuel costs, congestion..)

In [30]:
# Use the beta_1 value as five times its original
beta_1 = beta*0.6
beta_2 = beta*0.7
beta_3 = beta*0.8
beta_4 = beta*0.9
beta_5 = beta*1.1
beta_6 = beta*1.2
beta_7 = beta*1.3
beta_8 = beta*1.4

In [31]:
# Get the balancing factors. NB Setting of new field names for Ai and Bj.
df1 = balance_doubly_constrained(df,'Origin','Destination','O_i','D_j','Distance',beta,'exponential','Ai_exp','Bj_exp')
# Now predict the model again using the new Ai and Dj fields.
df1['Tra_1'] = np.round(df['O_i'] * df['Ai_exp'] * df['D_j'] * df['Bj_exp'] * np.exp(df['Distance']*beta_1))
# Check out the matrix
#pd.pivot_table(df,values='SIM_est_exp2',index ='Origin',columns='Destination',fill_value=0,aggfunc=sum,margins=True)
df1.head()

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


,Trips,Origin,Destination,OPop,DEmpl,Distance,log_OPop,log_DEmpl,log_Distance,O_i,...,attrsimFitted,doubsimfitted,doubsimfitted1,Ai_new,Bj_new,SIM_est_pow,Ai_exp,Bj_exp,SIM_est_exp,Tra_0.6
0,10728,Gamle Oslo,Grünerløkka,41736,29560,2143.111352,10.639119,10.294177,7.670014,69093,...,11043.0,11400.0,8995.0,0.088423,0.705430,11400.0,0.000003,0.810296,8995.0,8074.0
1,3820,Gamle Oslo,Sagene,41736,19408,3837.411674,10.639119,9.873441,8.252553,69093,...,3341.0,3197.0,4087.0,0.088423,0.619157,3197.0,0.000003,0.793952,4087.0,4316.0
2,7288,Gamle Oslo,St. Hanshaugen,41736,54104,4360.828507,10.639119,10.898663,8.380417,69093,...,5488.0,5920.0,6821.0,0.088423,0.843713,5920.0,0.000003,0.886683,6821.0,7211.0
3,6342,Gamle Oslo,Frogner,41736,48082,5157.307531,10.639119,10.780663,8.548170,69093,...,5332.0,5510.0,6403.0,0.088423,1.003974,5510.0,0.000003,0.981354,6403.0,6976.0
4,2404,Gamle Oslo,Ullern,41736,38334,5495.850695,10.639119,10.554093,8.611749,69093,...,2798.0,2554.0,3091.0,0.088423,1.088645,2554.0,0.000003,1.055214,3091.0,3327.0


In [ ]:
data['Tra_1']=df5['Tra_1']
data.to_excel('ML_Dist.xlsx')

In [32]:
df2 = balance_doubly_constrained(df,'Origin','Destination','O_i','D_j','Distance',beta_2,'exponential','Ai_exp','Bj_exp')
# Now predict the model again using the new Ai and Dj fields.
df2['Tra_0.7'] = np.round(df['O_i'] * df['Ai_exp'] * df['D_j'] * df['Bj_exp'] * np.exp(df['Distance']*beta_2))

df3 = balance_doubly_constrained(df,'Origin','Destination','O_i','D_j','Distance',beta_3,'exponential','Ai_exp','Bj_exp')
# Now predict the model again using the new Ai and Dj fields.
df3['Tra_0.8'] = np.round(df['O_i'] * df['Ai_exp'] * df['D_j'] * df['Bj_exp'] * np.exp(df['Distance']*beta_3))

df4 = balance_doubly_constrained(df,'Origin','Destination','O_i','D_j','Distance',beta_4,'exponential','Ai_exp','Bj_exp')
# Now predict the model again using the new Ai and Dj fields.
df4['Tra_0.9'] = np.round(df['O_i'] * df['Ai_exp'] * df['D_j'] * df['Bj_exp'] * np.exp(df['Distance']*beta_4))

df5 = balance_doubly_constrained(df,'Origin','Destination','O_i','D_j','Distance',beta_5,'exponential','Ai_exp','Bj_exp')
# Now predict the model again using the new Ai and Dj fields.
df5['Tra_1.1'] = np.round(df['O_i'] * df['Ai_exp'] * df['D_j'] * df['Bj_exp'] * np.exp(df['Distance']*beta_5))

df6 = balance_doubly_constrained(df,'Origin','Destination','O_i','D_j','Distance',beta_6,'exponential','Ai_exp','Bj_exp')
# Now predict the model again using the new Ai and Dj fields.
df6['Tra_1.2'] = np.round(df['O_i'] * df['Ai_exp'] * df['D_j'] * df['Bj_exp'] * np.exp(df['Distance']*beta_6))

df7 = balance_doubly_constrained(df,'Origin','Destination','O_i','D_j','Distance',beta_7,'exponential','Ai_exp','Bj_exp')
# Now predict the model again using the new Ai and Dj fields.
df7['Tra_1.3'] = np.round(df['O_i'] * df['Ai_exp'] * df['D_j'] * df['Bj_exp'] * np.exp(df['Distance']*beta_7))

df8 = balance_doubly_constrained(df,'Origin','Destination','O_i','D_j','Distance',beta_8,'exponential','Ai_exp','Bj_exp')
# Now predict the model again using the new Ai and Dj fields.
df8['Tra_1.4'] = np.round(df['O_i'] * df['Ai_exp'] * df['D_j'] * df['Bj_exp'] * np.exp(df['Distance']*beta_8))

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8


In [33]:
data['Tra_0.6']=df1['Tra_0.6']
data['Tra_0.7']=df2['Tra_0.7']
data['Tra_0.8']=df3['Tra_0.8']
data['Tra_0.9']=df4['Tra_0.9']
data['Tra_1.1']=df5['Tra_1.1']
data['Tra_1.2']=df6['Tra_1.2']
data['Tra_1.3']=df7['Tra_1.3']
data['Tra_1.4']=df8['Tra_1.4']
data.head()

,Trips,Origin,Destination,OPop,DEmpl,Distance,Tra_0.6,Tra_0.7,Tra_0.8,Tra_0.9,Tra_1.1,Tra_1.2,Tra_1.3,Tra_1.4
0,10728,Gamle Oslo,Grünerløkka,41736,29560,2143.111352,8074.0,8284.0,8508.0,8745.0,9256.0,9526.0,9804.0,10088.0
1,3820,Gamle Oslo,Sagene,41736,19408,3837.411674,4316.0,4256.0,4199.0,4143.0,4030.0,3972.0,3912.0,3850.0
2,7288,Gamle Oslo,St. Hanshaugen,41736,54104,4360.828507,7211.0,7106.0,7007.0,6912.0,6731.0,6643.0,6555.0,6466.0
3,6342,Gamle Oslo,Frogner,41736,48082,5157.307531,6976.0,6833.0,6691.0,6547.0,6258.0,6113.0,5966.0,5818.0
4,2404,Gamle Oslo,Ullern,41736,38334,5495.850695,3327.0,3280.0,3225.0,3162.0,3013.0,2930.0,2841.0,2747.0


In [ ]:
compare=df[["Origin", "Destination", "Distance", "Trips", "doubsimfitted1", "SIM_est_exp2"]]
compare.rename(columns={'doubsimfitted1': 'DistanceDecay', 'SIM_est_exp2': 'CostScenario'}, inplace = True)
compare.sort_values(by=["Distance"], inplace = True)
compare.set_index('Distance',inplace=True)
compare.head()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(compare.Trips, label='Actual Trips')
plt.plot(compare.DistanceDecay, label='Modelled Trips')
#plt.plot(compare.CostScenario, label='Cost SimScenario')
plt.xlabel('Distance')
plt.ylabel('Number of Trips')
plt.legend()
rcParams['figure.figsize'] = 16,12
fig

### Employment at 'Gamle Oslo' Urban District decreased by 50%

In [ ]:
whatif02_oslo = df
df.loc[(df['Origin'] == 'Marka') & (df['OriPop19'] <= 500000) & (df['Destination'] == 'Gamle Oslo')]

In [ ]:
def no_jobs(row):
    if row["Destination"] == "Gamle Oslo":
        val = 21560
    else:
        val = row["DestEmp19"]
    return val
        
whatif02_oslo["NewDestEmp19"] = whatif02_oslo.apply(no_jobs, axis =1)
whatif02_oslo.head()

In [ ]:
whatif02_oslo["prodsimest2"] = np.exp(whatif02_oslo["alpha_i"]+gamma*np.log(whatif02_oslo["NewDestEmp19"]) + beta*whatif02_oslo["log_Distance"])
whatif02_oslo["prodsimest2"] = round(whatif02_oslo["prodsimest2"],0)
#now we can convert the pivot table into a matrix
odata2 = whatif02_oslo.pivot_table(values ="prodsimest2", index="Origin", columns = "Destination", aggfunc=np.sum, margins=True)
odata2

In [ ]:
#calculate some new wj^alpha and d_ij^beta values
Dj2_gamma = whatif02_oslo["DestEmp19"]**gamma
dist_beta = whatif02_oslo["Distance"]**beta
#calcualte the first stage of the Ai values
whatif02_oslo["Ai1"] = Dj2_gamma * dist_beta
#now do the sum over all js bit
A_i = pd.DataFrame(whatif02_oslo.groupby(["Origin"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i"}, inplace=True)
#and write the A_i values back into the dataframe
whatif02_oslo = whatif02_oslo.merge(A_i, left_on="Origin", right_index=True, how="left")
#to check everything works, recreate the original estimates
whatif02_oslo["prodsimest3"] = whatif02_oslo["A_i"]*whatif02_oslo["O_i"]*Dj2_gamma*dist_beta
#round
whatif02_oslo["prodsimest3"] = round(whatif02_oslo["prodsimest3"])
#check
whatif02_oslo['emptrips_var'] = (whatif02_oslo['prodsimest1'] - whatif02_oslo['prodsimest3'])
whatif02_oslo

In [ ]:
#calculate some new wj^alpha and d_ij^beta values
Dj3_gamma = whatif02_oslo["NewDestEmp19"]**gamma
#calcualte the first stage of the Ai values
whatif02_oslo["Ai1"] = Dj3_gamma * dist_beta

#now do the sum over all js bit
A_i = pd.DataFrame(whatif02_oslo.groupby(["Origin"])["Ai1"].agg(np.sum))

A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i2"}, inplace=True)

#and write the A_i values back into the dataframe
whatif02_oslo = whatif02_oslo.merge(A_i, left_on="Origin", right_index=True, how="left")
#to check everything works, recreate the original estimates
whatif02_oslo["prodsimest4"] = whatif02_oslo["A_i2"]*whatif02_oslo["O_i"]*Dj3_gamma*dist_beta
#round
whatif02_oslo["prodsimest4"] = round(whatif02_oslo["prodsimest4"])
whatif02_oslo["varySIM02"] = whatif02_oslo["Trips"] - whatif02_oslo["prodsimest4"]
whatif02_oslo[["Origin", "Destination", "Trips", "prodsimest4", "varySIM02"]]
whatif02_oslo

### Population at 'Gamle Oslo' Urban District increased by 25%

In [ ]:
whatif01_oslo = df
df.loc[(df['Origin'] == 'Gamle Oslo') & (df['OriPop19'] <= 500000) & (df['Destination'] == 'Marka')]

In [ ]:
def new_pop(row):
    if row["Origin"] == "Gamle Oslo":
        val = 49670
    else:
        val = row["OriPop19"]
    return val
        
whatif01_oslo["OriPop19"] = whatif01_oslo.apply(new_pop, axis =1)
whatif01_oslo.head()